<a href="https://colab.research.google.com/github/prof-atritiack/visao-computacional-huggingface/blob/main/003_HuggingFace_Visao_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Hugging Face - Visão Computacional no Colab**

Este notebook apresenta exemplos de **classificação de imagens** e **detecção de objetos** utilizando modelos pré-treinados disponíveis no **Hugging Face Hub**.  
Inclui também chamadas à **Inference API** para executar tarefas diretamente na nuvem.



## **1. Instalação das dependências**

In [ ]:
!pip install -U huggingface_hub transformers pillow torch

## **2. Login no Hugging Face Hub**

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# O token deve ser configurado em: Colab -> Secrets -> HF_TOKEN
token = userdata.get('HF_TOKEN')
login(token=token)

## **3. Preparação das imagens de teste**

In [ ]:
from pathlib import Path
from PIL import Image

IMAGES_DIR = Path("datasets/imagens")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

exemplo_imagem = IMAGES_DIR / "exemplo.jpg"

if exemplo_imagem.exists():
    print("Imagem encontrada:", exemplo_imagem)
else:
    print("⚠️ Adicione uma imagem chamada 'exemplo.jpg' em datasets/imagens/")

## **4. Classificação de imagens**

In [ ]:
from transformers import pipeline
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

MODEL_CLF = "google/vit-base-patch16-224"
clf = pipeline(task="image-classification", model=MODEL_CLF)

if exemplo_imagem.exists():
    img = Image.open(exemplo_imagem).convert("RGB")
    preds = clf(str(exemplo_imagem))

    print("Resultados da classificação:")
    for p in preds:
        print(f"{p['label']}: {p['score']:.4f}")

    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv2_imshow(img_cv)

## **5. Detecção de objetos**

In [ ]:
from transformers import pipeline
from PIL import ImageDraw

MODEL_DET = "facebook/detr-resnet-50"
detector = pipeline(task="object-detection", model=MODEL_DET)

if exemplo_imagem.exists():
    img = Image.open(exemplo_imagem).convert("RGB")
    draw = ImageDraw.Draw(img)
    dets = detector(str(exemplo_imagem))

    print("Resultados da detecção:")
    cores = ["red", "blue", "green", "orange", "purple", "cyan", "magenta", "yellow"]
    for i, d in enumerate(dets):
        box = d['box']
        cor = cores[i % len(cores)]
        draw.rectangle([(box['xmin'], box['ymin']), (box['xmax'], box['ymax'])], outline=cor, width=3)
        draw.text((box['xmin'], box['ymin']-10), f"{d['label']} {d['score']:.2f}", fill=cor)
        print(f"{d['label']} ({d['score']:.2f}) -> {box}")

    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv2_imshow(img_cv)

## **6. Uso da Inference API (Classificação)**

In [ ]:
import requests, json

API_MODEL = MODEL_CLF
API_URL = f"https://api-inference.huggingface.co/models/{API_MODEL}"

headers = {"Authorization": f"Bearer {token}", "Content-Type": "image/jpeg"}

if exemplo_imagem.exists():
    with open(exemplo_imagem, "rb") as f:
        data = f.read()
    resp = requests.post(API_URL, headers=headers, data=data)

    print("Status da requisição:", resp.status_code)
    resposta = resp.json()
    print("Resposta da API (Classificação):")
    print(json.dumps(resposta, indent=2, ensure_ascii=False))

## **7. Uso da Inference API (Detecção de objetos)**

In [ ]:
API_MODEL = MODEL_DET
API_URL = f"https://api-inference.huggingface.co/models/{API_MODEL}"

if exemplo_imagem.exists():
    with open(exemplo_imagem, "rb") as f:
        data = f.read()
    resp = requests.post(API_URL, headers=headers, data=data)

    print("Status da requisição:", resp.status_code)
    resposta = resp.json()
    print("Resposta da API (Detecção):")
    print(json.dumps(resposta, indent=2, ensure_ascii=False))

    img = Image.open(exemplo_imagem).convert("RGB")
    draw = ImageDraw.Draw(img)
    cores = ["red", "blue", "green", "orange", "purple", "cyan", "magenta", "yellow"]

    if isinstance(resposta, list):
        for i, d in enumerate(resposta):
            if "box" in d:
                box = d["box"]
                cor = cores[i % len(cores)]
                draw.rectangle([(box["xmin"], box["ymin"]), (box["xmax"], box["ymax"])], outline=cor, width=3)
                draw.text((box["xmin"], box["ymin"]-10), f"{d['label']} {d['score']:.2f}", fill=cor)

    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv2_imshow(img_cv)